In [0]:
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import tensorflow as tf


class MNIST(object):
    def __init__(self, subset='train', batch_size=64, labeled_percent=0.2, shuffle=True):
        mnist = input_data.read_data_sets('data', one_hot=False)
        if subset == 'train':
            images = mnist.train._images
            labels = mnist.train._labels.reshape((-1, 1))
            np.random.seed(100)
            is_labeled = np.zeros((55000, 1))
            labeled_images = np.random.permutation(np.arange(55000))[:int(labeled_percent * 55000)]
            is_labeled[labeled_images] = 1
            np.random.seed()
        elif subset == 'valid':
            images = mnist.validation._images
            labels = mnist.validation._labels.reshape((-1, 1))
            is_labeled = np.ones((5000, 1))
        elif subset == 'test':
            images = mnist.test._images
            labels = mnist.test._labels.reshape((-1, 1))
            is_labeled = np.ones((10000, 1))
        else:
            raise NotImplementedError
        self._images = images
        self.images = self._images
        self._labels = labels
        self.labels = self._labels
        self._is_labeled = is_labeled
        self.is_labeled = self._is_labeled
        self.batch_size = batch_size
        self.num_samples = len(self.images)
        self.shuffle = shuffle
        if self.shuffle:
            self.shuffle_samples()
        self.next_batch_pointer = 0

    def shuffle_samples(self):
        image_indices = np.random.permutation(np.arange(self.num_samples))
        self.images = self._images[image_indices]
        self.labels = self._labels[image_indices]
        self.is_labeled = self._is_labeled[image_indices]

    def get_next_batch(self):
        num_samples_left = self.num_samples - self.next_batch_pointer
        if num_samples_left >= self.batch_size:
            x_batch = self.images[self.next_batch_pointer:self.next_batch_pointer + self.batch_size]
            y_batch = self.labels[self.next_batch_pointer:self.next_batch_pointer + self.batch_size]
            is_labeled_batch = self.is_labeled[self.next_batch_pointer:self.next_batch_pointer + self.batch_size]
            self.next_batch_pointer += self.batch_size
        else:
            x_partial_batch_1 = self.images[self.next_batch_pointer:self.num_samples]
            y_partial_batch_1 = self.labels[self.next_batch_pointer:self.num_samples]
            is_labeled_batch_1 = self.is_labeled[self.next_batch_pointer:self.num_samples]
            if self.shuffle:
                self.shuffle_samples()
            x_partial_batch_2 = self.images[0:self.batch_size - num_samples_left]
            y_partial_batch_2 = self.labels[0:self.batch_size - num_samples_left]
            is_labeled_batch_2 = self.is_labeled[0:self.batch_size - num_samples_left]
            x_batch = np.vstack((x_partial_batch_1, x_partial_batch_2))
            y_batch = np.vstack((y_partial_batch_1, y_partial_batch_2))
            is_labeled_batch = np.vstack((is_labeled_batch_1, is_labeled_batch_2))
            self.next_batch_pointer = self.batch_size - num_samples_left
        return x_batch, y_batch, is_labeled_batch

In [0]:
# TODO: (Optional)
# Define your convenient function for convolution, batch_norm,... (if applicable)


def AutoEncoder(input_tensor, is_training):
    # input_tensor: image tensor, similar to previous assignment, size [N, 784]
    # is_training: boolean tensor indicate whether if in training phase or testing phase

    # TODO: Define the architecture of your autoencoder here
    
    # return:
    # recon: reconstruction of the image by the autoencoder
    # logits: logits of the classification branch from the bottleneck of the autoencoder
    return recon, logits


In [4]:
def run():
    # General setup    
    EPOCHS = 10
    BATCH_SIZE = 64
    NUM_ITERS = int(55000 / BATCH_SIZE * EPOCHS)

    train_set = MNIST('train', batch_size=BATCH_SIZE)
    valid_set = MNIST('valid')
    test_set = MNIST('test', shuffle=False)

    tf.reset_default_graph()
    x = tf.placeholder(tf.float32, (None, 784))
    y = tf.placeholder(tf.int32, (None, 1))
    is_labeled = tf.placeholder(tf.float32, (None, 1))
    is_training = tf.placeholder(tf.bool, ())
    one_hot_y = tf.one_hot(y, 10)

    # create loss
    rate = 0.01
    recon, logits = AutoEncoder(x, is_training=is_training)
    prediction = tf.argmax(logits, axis=1)
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=one_hot_y) * is_labeled)
    recon_loss = tf.reduce_mean((recon - x) ** 2)
    loss_operation = cross_entropy + recon_loss
    optimizer = tf.train.AdamOptimizer(learning_rate=rate)
    grads_and_vars = optimizer.compute_gradients(loss_operation, tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES))
    training_operation = optimizer.apply_gradients(grads_and_vars)

    def evaluation(images, true_labels):
        eval_batch_size = 100
        predicted_labels = []
        for start_index in range(0, len(images), eval_batch_size):
            end_index = start_index + eval_batch_size
            batch_x = images[start_index: end_index]
            batch_predicted_labels = sess.run(prediction, feed_dict={x: batch_x, is_training: False})
            predicted_labels += list(batch_predicted_labels)
        predicted_labels = np.vstack(predicted_labels).flatten()
        true_labels = true_labels.flatten()
        accuracy = float((predicted_labels == true_labels).astype(np.int32).sum()) / len(images)
        return predicted_labels, accuracy

    # train
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    print("Training...")
    for i in range(NUM_ITERS):
        batch_x, batch_y, batch_is_labeled = train_set.get_next_batch()
        _ = sess.run(training_operation, feed_dict={x: batch_x, y: batch_y, is_labeled: batch_is_labeled, is_training: True})
        if (i + 1) % 1000 == 0 or i == NUM_ITERS - 1:
            _, validation_accuracy = evaluation(valid_set._images, valid_set._labels)
            print("Iter {}: Validation Accuracy = {:.3f}".format(i, validation_accuracy))

    print('Evaluating on test set')
    _, test_accuracy = evaluation(test_set._images, test_set._labels)
    print("Test Accuracy = {:.3f}".format(test_accuracy))

    sess.close()
    return test_accuracy


if __name__ == '__main__':
    run()

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
Instructions for updating:

Future major versi